In [85]:
#7/5/24 For the creation of an Autoencoder for Feature Extraction
#The goal is to create an autoencoder that can parse SMILES via one hot encoding or a graphical representation
#Main steps:
    #One hot encoding SMILES DONE
    #import into autoencoder
    #train to recreate SMILES
    #use latent space for feature prediction
#Mostly for testing, one hot data is too large to parse into jupyter
#Will test on drug target data

import pandas as pd
import numpy as np
#Data import
targetdata = pd.read_csv('/Users/james/Documents/Honours/Python/Target_Selection/Drug_Targets/DrugTargets.csv')

In [98]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [100]:
x_train.shape

(60000, 28, 28)

In [86]:
#Creating test/train splits
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(targetdata, test_size=0.2, random_state=81)
#Converts Toxicity Values to a list
ytrain = trainset['Toxicity_Values'].to_list
ytest = testset['Toxicity_Values'].to_list
#Converts encoded drug target values to an array
xtrain = trainset.iloc[:, 2:].values
xtest = testset.iloc[:, 2:].values

In [92]:
print(xtrain.shape)
print(xtrain[0])

(10595, 334)
[0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0]


In [88]:
#Determines layer size and input/output
#Here it turns the 334 one hot encoded features to 81
import tensorflow as tf
from tensorflow import keras

#Encoder Creation
enc_in = keras.Input(shape=(334, 1))
x = keras.layers.Flatten()(enc_in)
enc_out = keras.layers.Dense(42, activation="relu")(x)
encoder = keras.Model(enc_in, enc_out, name='encoder')

#Decoder Creation
decoder_in = keras.layers.Dense(42, activation='relu')(enc_out)
x = keras.layers.Dense(334, activation='relu')(decoder_in)
decoder_out = keras.layers.Reshape((334,1))(x)

#Optimiser
optimise = tf.keras.optimizers.legacy.Adam()

#Model Creation
autoencoder = keras.Model(enc_in, decoder_out, name='autoencoder')

autoencoder.compile(optimise, loss='binary_crossentropy')

In [101]:
autoencoder.summary()


Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 334, 1)]          0         
                                                                 
 flatten_10 (Flatten)        (None, 334)               0         
                                                                 
 dense_67 (Dense)            (None, 42)                14070     
                                                                 
 dense_68 (Dense)            (None, 42)                1806      
                                                                 
 dense_69 (Dense)            (None, 334)               14362     
                                                                 
 reshape_9 (Reshape)         (None, 334, 1)            0         
                                                                 
Total params: 30,238
Trainable params: 30,238
Non-train

In [104]:
xtrain1 = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))

In [108]:
epochs = 3
for epoch in range(epochs):
    
    logs = autoencoder.fit(
    xtrain,
    xtrain,
    epochs=1,
    batch_size=32, validation_split=0.2)
    
    
    autoencoder.save(str(epoch))
    

265/265 [==============================] - 0s 672us/step - loss: 0.1512 - val_loss: 0.1498
INFO:tensorflow:Assets written to: 0/assets


2024-05-13 15:48:37.391363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-13 15:48:37.438446: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
INFO:tensorflow:Assets written to: 0/assets


265/265 [==============================] - 0s 641us/step - loss: 0.1485 - val_loss: 0.1496
INFO:tensorflow:Assets written to: 1/assets


2024-05-13 15:48:37.772995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-13 15:48:37.819006: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
INFO:tensorflow:Assets written to: 1/assets


265/265 [==============================] - 0s 632us/step - loss: 0.1487 - val_loss: 0.1527
INFO:tensorflow:Assets written to: 2/assets


2024-05-13 15:48:38.143564: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
2024-05-13 15:48:38.189550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,334]
	 [[{{node inputs}}]]
INFO:tensorflow:Assets written to: 2/assets


In [109]:
epoch

2